# IPL2023-Powerplay-Score-Prediction

## Importing libraries and data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
def log(*args):
    print('👉', *args)

In [4]:
ball_by_ball_orig = pd.read_csv('./Data/IPL_Ball_by_Ball_2008_2022.csv')
matches_result_orig = pd.read_csv('./Data/IPL_Matches_Result_2008_2022.csv')

In [5]:
ball_by_ball_orig.shape
ball_by_ball_orig.head()

(225954, 17)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [6]:
matches_result_orig.shape
matches_result_orig.head()

(950, 20)

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan


## Data cleaning

In [7]:
# Renaming and selecting columns for ball_by_ball DataFrame
ball_by_ball = ball_by_ball_orig.rename(columns={
    'ID': 'match_id',
    'ballnumber': 'ball_number',
    'non-striker': 'non_striker',
    'BattingTeam': 'batting_team',
}).loc[:, [
    'match_id',
    'innings',
    'batting_team',
    'overs',
    'ball_number',
    'batter',
    'bowler',
    'total_run',
]]

# Renaming and selecting columns for matches_result DataFrame
matches_result = matches_result_orig.rename(columns={
    'ID': 'match_id',
    'Team1': 'team_1',
    'Team2': 'team_2',
    'Venue': 'venue',
}).loc[:, [
    'match_id',
    'team_1',
    'team_2',
    'venue',
]]

In [8]:
ball_by_ball.head()

,match_id,innings,batting_team,overs,ball_number,batter,bowler,total_run
0,1312200,1,Rajasthan Royals,0,1,YBK Jaiswal,Mohammed Shami,0
1,1312200,1,Rajasthan Royals,0,2,YBK Jaiswal,Mohammed Shami,1
2,1312200,1,Rajasthan Royals,0,3,JC Buttler,Mohammed Shami,1
3,1312200,1,Rajasthan Royals,0,4,YBK Jaiswal,Mohammed Shami,0
4,1312200,1,Rajasthan Royals,0,5,YBK Jaiswal,Mohammed Shami,0


In [9]:
matches_result.head()

,match_id,team_1,team_2,venue
0,1312200,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad"
1,1312199,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad"
2,1312198,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata"
3,1312197,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata"
4,1304116,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai"


## Team and venue names normalization

In [26]:
# https://www.iplt20.com/matches/schedule/men

venue_mapping_normal = {
    "Arun Jaitley Stadium": "Arun Jaitley Stadium",
    "Arun Jaitley Stadium, Delhi": "Arun Jaitley Stadium",
    "Feroz Shah Kotla": "Arun Jaitley Stadium",
    "Barsapara Cricket Stadium": "Barsapara Cricket Stadium",
    "Barsapara Cricket Stadium, Guwahati": "Barsapara Cricket Stadium",
    "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "Eden Gardens": "Eden Gardens",
    "Eden Gardens, Kolkata": "Eden Gardens",
    "Himachal Pradesh Cricket Association Stadium": "Himachal Pradesh Cricket Association Stadium",
    "Himachal Pradesh Cricket Association Stadium, Dharamsala": "Himachal Pradesh Cricket Association Stadium",
    "M Chinnaswamy Stadium": "M Chinnaswamy Stadium",
    "M Chinnaswamy Stadium, Bengaluru": "M Chinnaswamy Stadium",
    "M Chinnaswamy Stadium, Bangalore": "M Chinnaswamy Stadium",
    "M.Chinnaswamy Stadium": "M Chinnaswamy Stadium",
    "M.Chinnaswamy Stadium, Bengaluru": "M Chinnaswamy Stadium",
    "M.Chinnaswamy Stadium, Bangalore": "M Chinnaswamy Stadium",
    "MA Chidambaram Stadium": "MA Chidambaram Stadium",
    "MA Chidambaram Stadium, Chennai": "MA Chidambaram Stadium",
    "MA Chidambaram Stadium, Chepauk": "MA Chidambaram Stadium",
    "MA Chidambaram Stadium, Chepauk, Chennai": "MA Chidambaram Stadium",
    "Narendra Modi Stadium": "Narendra Modi Stadium",
    "Narendra Modi Stadium, Ahmedabad": "Narendra Modi Stadium",
    "Punjab Cricket Association IS Bindra Stadium": "Punjab Cricket Association IS Bindra Stadium",
    "Punjab Cricket Association IS Bindra Stadium, Mohali": "Punjab Cricket Association IS Bindra Stadium",
    "Punjab Cricket Association Stadium, Mohali": "Punjab Cricket Association IS Bindra Stadium",
    "Rajiv Gandhi International Stadium": "Rajiv Gandhi International Stadium",
    "Rajiv Gandhi International Stadium, Hyderabad": "Rajiv Gandhi International Stadium",
    "Rajiv Gandhi International Stadium, Uppal": "Rajiv Gandhi International Stadium",
    "Sawai Mansingh Stadium": "Sawai Mansingh Stadium",
    "Sawai Mansingh Stadium, Jaipur": "Sawai Mansingh Stadium",
    "Wankhede Stadium": "Wankhede Stadium",
    "Wankhede Stadium, Mumbai": "Wankhede Stadium"
}

In [27]:
venue_mapping_kebab = {
    "arun-jaitley-stadium": "Arun Jaitley Stadium",
    "arun-jaitley-stadium-delhi": "Arun Jaitley Stadium",
    "feroz-shah-kotla": "Arun Jaitley Stadium",
    "barsapara-cricket-stadium": "Barsapara Cricket Stadium",
    "barsapara-cricket-stadium-guwahati": "Barsapara Cricket Stadium",
    "bharat-ratna-shri-atal-bihari-vajpayee-ekana-cricket-stadium": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "bharat-ratna-shri-atal-bihari-vajpayee-ekana-cricket-stadium-lucknow": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "eden-gardens": "Eden Gardens",
    "eden-gardens-kolkata": "Eden Gardens",
    "himachal-pradesh-cricket-association-stadium": "Himachal Pradesh Cricket Association Stadium",
    "himachal-pradesh-cricket-association-stadium-dharamsala": "Himachal Pradesh Cricket Association Stadium",
    "m-chinnaswamy-stadium": "M Chinnaswamy Stadium",
    "m-chinnaswamy-stadium-bengaluru": "M Chinnaswamy Stadium",
    "m-chinnaswamy-stadium-bangalore": "M Chinnaswamy Stadium",
    "mchinnaswamy-stadium": "M Chinnaswamy Stadium",
    "mchinnaswamy-stadium-bengaluru": "M Chinnaswamy Stadium",
    "mchinnaswamy-stadium-bangalore": "M Chinnaswamy Stadium",
    "ma-chidambaram-stadium": "MA Chidambaram Stadium",
    "ma-chidambaram-stadium-chennai": "MA Chidambaram Stadium",
    "ma-chidambaram-stadium-chepauk": "MA Chidambaram Stadium",
    "ma-chidambaram-stadium-chepauk-chennai": "MA Chidambaram Stadium",
    "narendra-modi-stadium": "Narendra Modi Stadium",
    "narendra-modi-stadium-ahmedabad": "Narendra Modi Stadium",
    "punjab-cricket-association-is-bindra-stadium": "Punjab Cricket Association IS Bindra Stadium",
    "punjab-cricket-association-is-bindra-stadium-mohali": "Punjab Cricket Association IS Bindra Stadium",
    "punjab-cricket-association-stadium-mohali": "Punjab Cricket Association IS Bindra Stadium",
    "rajiv-gandhi-international-stadium": "Rajiv Gandhi International Stadium",
    "rajiv-gandhi-international-stadium-hyderabad": "Rajiv Gandhi International Stadium",
    "rajiv-gandhi-international-stadium-uppal": "Rajiv Gandhi International Stadium",
    "sawai-mansingh-stadium": "Sawai Mansingh Stadium",
    "sawai-mansingh-stadium-jaipur": "Sawai Mansingh Stadium",
    "wankhede-stadium": "Wankhede Stadium",
    "wankhede-stadium-mumbai": "Wankhede Stadium"
}

In [28]:
venue_mapping_tags = {
    "delhi": "Arun Jaitley Stadium",
    "arun jaitley": "Arun Jaitley Stadium",
    "guwahati": "Barsapara Cricket Stadium",
    "barsapara": "Barsapara Cricket Stadium",
    "bhupen hazarika": "Barsapara Cricket Stadium",
    "assam cricket association stadium": "Barsapara Cricket Stadium",
    "lucknow": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "ekana": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "atal bihari": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
    "kolkata": "Eden Gardens",
    "eden gardens": "Eden Gardens",
    "dharamsala": "Himachal Pradesh Cricket Association Stadium",
    "himachal pradesh": "Himachal Pradesh Cricket Association Stadium",
    "bengaluru": "M Chinnaswamy Stadium",
    "bengalore": "M Chinnaswamy Stadium",
    "chinnaswamy": "M Chinnaswamy Stadium",
    "chennai": "MA Chidambaram Stadium",
    "chepauk": "MA Chidambaram Stadium",
    "chidambaram": "MA Chidambaram Stadium",
    "ahmedabad": "Narendra Modi Stadium",
    "narendra modi": "Narendra Modi Stadium",
    "mohali": "Punjab Cricket Association IS Bindra Stadium",
    "punjab cricket association": "Punjab Cricket Association IS Bindra Stadium",
    "is bindra": "Punjab Cricket Association IS Bindra Stadium",
    "hyderabad": "Rajiv Gandhi International Stadium",
    "rajiv gandhi": "Rajiv Gandhi International Stadium",
    "jaipur": "Sawai Mansingh Stadium",
    "sawai mansingh": "Sawai Mansingh Stadium",
    "mumbai": "Wankhede Stadium",
    "wankhede": "Wankhede Stadium"
}

In [47]:
# venues in `matches_result` which aren't present in `venue_mapping_normal.keys`,
# collectively considered as `Other` venue after mapping
np.setdiff1d(matches_result.venue.unique(), list(venue_mapping_normal.keys()))

array(['Other'], dtype=object)

In [33]:
set(matches_result['team_1'].unique()) == set(
    matches_result['team_2'].unique()) == set(ball_by_ball['batting_team'].unique() == set(ball_by_ball['batting_team'].unique())

True

In [41]:
team_mapping = {
    'Rajasthan Royals': 'Rajasthan Royals',
    'Gujarat Titans': 'Gujarat Titans',
    'Royal Challengers Bangalore': 'Royal Challengers Bangalore',
    'Lucknow Super Giants': 'Lucknow Super Giants',
    'Sunrisers Hyderabad': 'Sunrisers Hyderabad',
    'Mumbai Indians': 'Mumbai Indians',
    'Chennai Super Kings': 'Chennai Super Kings',
    'Kolkata Knight Riders': 'Kolkata Knight Riders',
    'Kings XI Punjab': 'Punjab Kings',
    'Punjab Kings': 'Punjab Kings',
    'Delhi Daredevils': 'Delhi Capitals',
    'Delhi Capitals': 'Delhi Capitals',
}

In [48]:
# teams in `team_mapping.keys` which aren't present in `matches_result`
# before mapping: [], after mapping: ['Delhi Daredevils', 'Kings XI Punjab']
np.setdiff1d(
    list(team_mapping.keys()), matches_result['team_1'].unique()
)

array(['Delhi Daredevils', 'Kings XI Punjab'], dtype='<U27')

In [49]:
# teams in `matches_result` which aren't present in `team_mapping.keys`,
# collectively considered as `Other` team after mapping
np.setdiff1d(
    matches_result['team_1'].unique(), list(team_mapping.keys())
)

array(['Other'], dtype=object)

## Apply team and venue mapping

In [44]:
matches_result.venue = matches_result.venue.map(
    venue_mapping_normal).fillna('Other')

In [45]:
matches_result.team_1 = matches_result.team_1.map(team_mapping).fillna('Other')
matches_result.team_2 = matches_result.team_2.map(team_mapping).fillna('Other')

In [46]:
ball_by_ball.batting_team = ball_by_ball.batting_team.map(
    team_mapping).fillna('Other')

## Mapping verification

In [69]:
# new venues in ipl23: ['Barsapara Cricket Stadium', 'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium']

list_of_ipl23_venues = ['Arun Jaitley Stadium', 'Barsapara Cricket Stadium',
                        'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium',
                        'Eden Gardens', 'Himachal Pradesh Cricket Association Stadium',
                        'M Chinnaswamy Stadium', 'MA Chidambaram Stadium',
                        'Narendra Modi Stadium',
                        'Punjab Cricket Association IS Bindra Stadium',
                        'Rajiv Gandhi International Stadium', 'Sawai Mansingh Stadium',
                        'Wankhede Stadium']

list_of_ipl23_teams = ['Chennai Super Kings', 'Delhi Capitals', 'Gujarat Titans',
                       'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians',
                       'Punjab Kings', 'Rajasthan Royals', 'Royal Challengers Bangalore',
                       'Sunrisers Hyderabad']

In [70]:
# list_of_ipl23_venues == venue_mapping_normal
np.array_equal(np.unique(
    list(venue_mapping_normal.values())
), list_of_ipl23_venues)

# list_of_ipl23_teams == team_mapping
np.array_equal(np.unique(
    list(team_mapping.values())
), list_of_ipl23_teams)

True

True

## Normalize overs and innings in `ball_by_ball`

In [76]:
ball_by_ball['overs'].unique()
ball_by_ball['innings'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19], dtype=int64)

array([1, 2, 3, 4, 5, 6], dtype=int64)

In [88]:
ball_by_ball = ball_by_ball.loc[(
    ball_by_ball.overs <= 5) & (ball_by_ball.innings <= 2)]

## ⭐ Integration of `ball_by_ball` and `matches_result`

In [91]:
ball_by_ball

,match_id,innings,batting_team,overs,ball_number,batter,bowler,total_run
0,1312200,1,Rajasthan Royals,0,1,YBK Jaiswal,Mohammed Shami,0
1,1312200,1,Rajasthan Royals,0,2,YBK Jaiswal,Mohammed Shami,1
2,1312200,1,Rajasthan Royals,0,3,JC Buttler,Mohammed Shami,1
3,1312200,1,Rajasthan Royals,0,4,YBK Jaiswal,Mohammed Shami,0
4,1312200,1,Rajasthan Royals,0,5,YBK Jaiswal,Mohammed Shami,0
...,...,...,...,...,...,...,...,...
225886,335982,2,Royal Challengers Bangalore,5,2,W Jaffer,AB Dinda,0
225887,335982,2,Royal Challengers Bangalore,5,3,MV Boucher,AB Dinda,0
225888,335982,2,Royal Challengers Bangalore,5,4,MV Boucher,AB Dinda,1
225889,335982,2,Royal Challengers Bangalore,5,5,CL White,AB Dinda,1


In [92]:
matches_result

,match_id,team_1,team_2,venue
0,1312200,Rajasthan Royals,Gujarat Titans,Narendra Modi Stadium
1,1312199,Royal Challengers Bangalore,Rajasthan Royals,Narendra Modi Stadium
2,1312198,Royal Challengers Bangalore,Lucknow Super Giants,Eden Gardens
3,1312197,Rajasthan Royals,Gujarat Titans,Eden Gardens
4,1304116,Sunrisers Hyderabad,Punjab Kings,Wankhede Stadium
...,...,...,...,...
945,335986,Kolkata Knight Riders,Other,Eden Gardens
946,335985,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium
947,335984,Delhi Capitals,Rajasthan Royals,Arun Jaitley Stadium
948,335983,Punjab Kings,Chennai Super Kings,Punjab Cricket Association IS Bindra Stadium
